# load model


In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
#os.environ["CUDA_VISIBLE_DEVICES"] = "1" # select GPU
sys.path.append('/home/chenghao/Project/cav-mae')

# cav-mae 的整体模型定义并导入预训练权重

In [ ]:
import os
import torch
import timm
from src.models import CAVMAE

assert timm.__version__ == '0.4.5' # it is important to have right version of timm

model_path = 'pretrained_model/audio_model.pth'
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7" # select GPU
# CAV-MAE model with decoder
#print(torch.cuda.device_count())
torch.cuda.set_device(1)
audio_model = CAVMAE(audio_length=1024,
                     modality_specific_depth=11,
                     norm_pix_loss=True, tr_pos=False) # most models are trained with pixel normalization and non-trainabe positional embedding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mdl_weight = torch.load(model_path, map_location=device)
audio_model = torch.nn.DataParallel(audio_model) # it is important to convert the model to dataparallel object as all weights are saved in dataparallel format (i.e., in module.xxx)
miss, unexpected = audio_model.load_state_dict(mdl_weight, strict=False)
print('miss=',miss, 'unexpected=', unexpected) # check if all weights are correctly loaded

# 定义my_cav-mae模型，查看其中的video模块输出维度

In [12]:
import os
import torch
import timm
from src.models.my_model import MY_MODULE, MY_FT

pretrained_mae_path = 'pretrained_model/pretrained_maedfer.pth'
my_model = MY_FT(label_dim = 11, img_size=224, audio_length=1024, 
                       modality_specific_depth=11,
                       norm_pix_loss=True, tr_pos=False, pretrained_mae_path=pretrained_mae_path)


Use norm_pix_loss:  True
Number of Audio Patches: 512, Visual Patches: 196
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 196, 768])


In [13]:

Video_data = torch.randn(8, 3, 8, 224, 224)
# 从给定文件夹下面取所有的8帧，制作一个videodata
Audio_data = torch.randn(8, 1024, 128)
output = my_model(Audio_data, Video_data,'multimodal')

In [14]:
print(output)
print(output.shape)

tensor([[ 5.2988e-01, -9.9112e-01, -3.3496e+00, -1.6574e+00, -1.6092e-01,
         -2.4413e+00,  1.2220e+00,  1.7428e+00, -1.4540e+00,  1.7196e-01,
         -1.9195e+00],
        [ 6.3494e-01, -1.1141e+00, -3.4143e+00, -1.7632e+00,  4.5381e-02,
         -2.3578e+00,  1.1923e+00,  1.6898e+00, -1.5633e+00,  1.0128e-01,
         -1.9564e+00],
        [ 6.1013e-01, -1.2794e+00, -3.4199e+00, -1.7083e+00, -1.9097e-01,
         -2.3626e+00,  1.2999e+00,  1.8093e+00, -1.4972e+00,  7.2735e-02,
         -1.7189e+00],
        [ 6.9055e-01, -1.2137e+00, -3.4993e+00, -1.6629e+00, -1.2731e-01,
         -2.4378e+00,  1.2881e+00,  1.6241e+00, -1.3302e+00,  4.7066e-02,
         -1.9907e+00],
        [ 5.9498e-01, -1.1333e+00, -3.3223e+00, -1.7042e+00, -1.8942e-02,
         -2.4774e+00,  1.1405e+00,  1.7786e+00, -1.3530e+00,  1.2421e-01,
         -1.7782e+00],
        [ 6.3174e-01, -1.0541e+00, -3.5229e+00, -1.5947e+00, -2.6083e-01,
         -2.4845e+00,  1.1276e+00,  1.6438e+00, -1.5168e+00,  1.5932e-0

# 定义mae dfer模型

In [13]:
#from src.models import modeling_finetune
import torch
from src.models.modeling_finetune import vit_base_patch16_160 ,vit_base_dim512_no_depth_patch16_160

# 定义模型
model1 = vit_base_dim512_no_depth_patch16_160(num_classes=11)
model1 = model1.to('cuda')
pretrained_model = '/home/chenghao/Project/cav-mae/pretrained_model/pretrained_maedfer.pth'
ckt = torch.load(pretrained_model, map_location='cpu')
print(ckt.keys())

msg = model1.load_state_dict(ckt['model'], strict=False)
print(msg)

dict_keys(['model', 'optimizer', 'epoch', 'scaler', 'args'])
_IncompatibleKeys(missing_keys=['patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.q_bias', 'blocks.0.attn.v_bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.q_bias', 'blocks.1.attn.v_bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.q_bias', 'blocks.2.attn.v_bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.n

# 用maedfer 进行一次前向

In [14]:

# 创建随机输入数据
input_data = torch.rand(8, 3, 16, 160, 160)

# 进行前向传播
input_data = input_data.to('cuda:0')
output1, output1_feature = model1(input_data,save_feature=True)
model1.reset_classifier( num_classes=10)
print(output1_feature.shape)
print(output1.shape)


torch.Size([8, 3, 16, 160, 160])
torch.Size([8, 3, 16, 160, 160])
torch.Size([8, 512])
torch.Size([8, 11])


In [ ]:
from src.models.modeling_pretrain import PretrainVisionTransformer
model2 = PretrainVisionTransformer()
input_data = torch.rand(8, 3, 16, 224, 224)
input_mask = torch.rand(8, 16, 160, 160)
output2, output2_feature = model2(input_data, input_mask)